In [1]:
import pandas as pd
import numpy as np
import spacy
from textblob import TextBlob

In [12]:
metadata = pd.read_csv('metadatabook2.csv')

In [13]:
metadata

,Filename,PlaceID,PlaceName,Date,Coordinates,Text
0,Letter 1,1.0,Lake Tahoe,02-09-1873,"39.0885405,-120.0503528",Letter I\nLake Tahoe—Morning in San Francisco—...
1,Letter 2,2.0,"Cheyenne, Wyoming",07-09-1873,"41.139981,-104.820246","Letter II\nA lady's ""get-up""—Grizzly bears—The..."
2,Letter 3.1,2.0,"Cheyenne, Wyoming",08-09-1873,"41.139981,-104.820246","A Temple of Morpheus—Utah—A ""God-forgotten"" to..."
3,Letter 3.2,2.0,"Cheyenne, Wyoming",09-09-1873,"41.139981,-104.820246",September 9.\n\nI have found at the post offic...
4,Letter 3.3,3.0,Fort Collins,10-09-1873,"40.5871782,-105.0770113","FORT COLLINS, September 10.\n\nIt gave me a st..."
5,Letter 4.1,4.0,"Canyon, Colorado",12-09-1873,"40.3919856,-105.4327321",A plague of flies—A melancholy charioteer—The ...
6,Letter 4.2,4.0,"Canyon, Colorado",16-09-1873,"40.3919856,-105.4327321","September 16.\n\nFive days here, and I am no n..."
7,Letter 5.1,4.0,"Canyon, Colorado",19-09-1873,"40.3919856,-105.4327321","\n\n\nA dateless day—""Those hands of yours""—A..."
8,Letter 5.2,NaN,Nameless region,20-09-1873,NaN,"NAMELESS REGION, ROCKY MOUNTAINS, September.\n..."
9,Letter 5.3,5.0,"Lower Canyon, Colorado",21-09-1873,"40.3806706,-105.1923793","LOWER CANYON, September 21.\n\nWe never reache..."


In [14]:
import re

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Handle missing or non-string values
    if not isinstance(text, str):
        return ""  # Return an empty string for invalid entries
    # Clean the text to remove special characters, numbers, and extra spaces
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    # Tokenize and lemmatize the text
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop]  # Remove stopwords and lemmatize
    return ' '.join(tokens)

# Apply preprocessing to the dataset
metadata['Processed_Content'] = metadata['Text'].apply(preprocess_text)

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [15]:
metadata['Text'] = metadata['Text'].fillna("").astype(str)

metadata['polarity'] = metadata.Text.apply(lambda x: TextBlob(x).polarity)
metadata['subjectivity'] = metadata.Text.apply(lambda x: TextBlob(x).subjectivity)
metadata['sentiment'] = np.where(metadata.polarity > 0, 'positive',
                                 np.where(metadata.polarity < 0, 'negative', 'neutral'))

output_path = "textblob_results.csv"
metadata.to_csv(output_path, index=False)

metadata.head()



,Filename,PlaceID,PlaceName,Date,Coordinates,Text,Processed_Content,polarity,subjectivity,sentiment
0,Letter 1,1.0,Lake Tahoe,02-09-1873,"39.0885405,-120.0503528",Letter I\nLake Tahoe—Morning in San Francisco—...,letter \n lake tahoemorne san franciscodusta p...,0.097865,0.467918,positive
1,Letter 2,2.0,"Cheyenne, Wyoming",07-09-1873,"41.139981,-104.820246","Letter II\nA lady's ""get-up""—Grizzly bears—The...",letter ii \n ladys getupgrizzly bearsthe gem s...,0.064370,0.474719,positive
2,Letter 3.1,2.0,"Cheyenne, Wyoming",08-09-1873,"41.139981,-104.820246","A Temple of Morpheus—Utah—A ""God-forgotten"" to...",temple morpheusutaha godforgotten towna distre...,0.022578,0.485667,positive
3,Letter 3.2,2.0,"Cheyenne, Wyoming",09-09-1873,"41.139981,-104.820246",September 9.\n\nI have found at the post offic...,september \n\n find post office circular lette...,0.011351,0.490230,positive
4,Letter 3.3,3.0,Fort Collins,10-09-1873,"40.5871782,-105.0770113","FORT COLLINS, September 10.\n\nIt gave me a st...",fort collin september \n\n give strange sensat...,0.076605,0.486954,positive


In [16]:
print(metadata['sentiment'].value_counts())

sentiment
positive    43
negative     6
Name: count, dtype: int64
